In [1]:
import re, os, sys, json, time, requests
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
# TODO load config.json, load eval.parquet
fs = sorted(Path('../outputs').glob('**/eval.parquet'))
fs

[PosixPath('../outputs/alpaca_low_quality-Qwen3-0.6B-sft/wassname-Qwen3-06B-sft_dpo_alpaca_low_quality/2025-06-12_20-42-34/eval.parquet'),
 PosixPath('../outputs/alpaca_low_quality-Qwen3-0.6B-sft/wassname-Qwen3-06B-sft_hs-None-InnerDPO_alpaca_low_quality/2025-06-12_20-15-08/eval.parquet'),
 PosixPath('../outputs/alpaca_mmlu-Qwen3-0.6B-sft/wassname-Qwen3-06B-sft_dpo_alpaca_mmlu/2025-06-12_16-01-44/eval.parquet'),
 PosixPath('../outputs/alpaca_mmlu-Qwen3-0.6B-sft/wassname-Qwen3-06B-sft_dpo_alpaca_mmlu/2025-06-12_21-38-18/eval.parquet'),
 PosixPath('../outputs/alpaca_mmlu-Qwen3-0.6B-sft/wassname-Qwen3-06B-sft_hs-None-InnerDPO_alpaca_mmlu/2025-06-12_15-34-58/eval.parquet'),
 PosixPath('../outputs/alpaca_mmlu-Qwen3-0.6B-sft/wassname-Qwen3-06B-sft_hs-None-InnerDPO_alpaca_mmlu/2025-06-12_21-09-38/eval.parquet'),
 PosixPath('../outputs/code_easy-Qwen3-0.6B-sft/wassname-Qwen3-06B-sft_dpo_code_easy/2025-06-12_17-03-31/eval.parquet'),
 PosixPath('../outputs/code_easy-Qwen3-0.6B-sft/wassname-Qwen3

In [3]:
def load_config_df(f):
    print(f.parent.parent.stem)
    config = json.load((f.parent / 'config.json').open())
    config.update(config.pop('post'))
    # log = (f.parent / 'log.txt').open().read()

    # dataframe of all non list/dict/tuple in config
    config_df = pd.DataFrame({k: v for k, v in config.items() if not isinstance(v, (list, dict, tuple))}, index=[0])
    # config_df['log'] = log
    config_df['file'] = f
    config_df['ts'] = f.parent.stem
    _, method, _ = f.parent.parent.stem.split('_', 2) # this is {model}_{method}_{dataset}
    config_df['method'] = method

    return config_df


df = pd.concat([load_config_df(f) for f in fs])
df.columns

# load_config(fs[0])

wassname-Qwen3-06B-sft_dpo_alpaca_low_quality
wassname-Qwen3-06B-sft_hs-None-InnerDPO_alpaca_low_quality
wassname-Qwen3-06B-sft_dpo_alpaca_mmlu
wassname-Qwen3-06B-sft_dpo_alpaca_mmlu
wassname-Qwen3-06B-sft_hs-None-InnerDPO_alpaca_mmlu
wassname-Qwen3-06B-sft_hs-None-InnerDPO_alpaca_mmlu
wassname-Qwen3-06B-sft_dpo_code_easy
wassname-Qwen3-06B-sft_hs-None-InnerDPO_code_easy
wassname-Qwen3-06B-sft_dpo_cooking
wassname-Qwen3-06B-sft_hs-None-InnerDPO_cooking
princeton-nlp-Llama-3-Base-8B-SFT_side-None-InnerDPO_math
allenai-OLMo-2-0425-1B-SFT_side-None-InnerDPO_math
wassname-Qwen3-06B-sft_side-None-InnerDPO_math


Index(['lr', 'weight_decay', 'gradient_clip_val', 'ideal_batch_size',
       'pl_precision', 'num_workers', 'dataset', 'verbose', 'seed', 'patience',
       'dev', 'load_in_4bit', 'load_in_8bit', 'use_grad_paging', 'n_samples',
       'eval_samples', 'max_length', 'max_prompt_length', 'base_model',
       'batch_size', 'save', 'wandb', 'use_policy_weights', 'dpo_agg_type',
       'β', 'group_name', 'adapter_name', 'human_name', 'short_name',
       'long_name', 'model_fname', 'ds_name_train', 'run_fname', 'save_dir',
       'ts', 'file', 'method', 'collection_layers', 'collect_input',
       'collect_hs', 'use_wpo'],
      dtype='object')

In [4]:
def load_config(f):
    print(f.parent.parent.stem)
    config = json.load((f.parent / 'config.json').open())
    # log = (f.parent / 'log.txt').open().read()

    # dataframe of all non list/dict/tuple in config
    config['file'] = f
    config['ts'] = f.parent.stem
    _, method, _ = f.parent.parent.stem.split('_', 2) # this is {model}_{method}_{dataset}
    config['method'] = method

    return config


# df = pd.concat([load_config(f) for f in fs])
# df.columns

load_config(fs[0])

wassname-Qwen3-06B-sft_dpo_alpaca_low_quality


{'lr': 2e-05,
 'weight_decay': 0.01,
 'gradient_clip_val': 10.0,
 'ideal_batch_size': 32,
 'pl_precision': 'bf16-mixed',
 'num_workers': 8,
 'dataset': 'alpaca_low_quality',
 'verbose': 2,
 'seed': 1,
 'patience': 3,
 'dev': False,
 'load_in_4bit': False,
 'load_in_8bit': False,
 'use_grad_paging': False,
 'n_samples': 25000,
 'eval_samples': 750,
 'max_length': 512,
 'max_prompt_length': 450,
 'base_model': 'wassname/Qwen3-0.6B-sft',
 'batch_size': 18,
 'save': True,
 'wandb': True,
 'use_policy_weights': False,
 'dpo_agg_type': 'ipo',
 'β': 0.1,
 'post': {'group_name': 'alpaca_low_quality-Qwen3-0.6B-sft',
  'adapter_name': 'dpo',
  'human_name': 'Dpo dataset=alpaca_low_quality',
  'short_name': 'Dpo dataset=AlpacLo',
  'long_name': 'base_model=wassname/Qwen3-0.6B-sft batch_size=18 dataset=alpaca_low_quality seed=1 verbose=2',
  'model_fname': 'wassname-Qwen3-06B-sft_dpo_alpaca_low_quality',
  'ds_name_train': 'alpaca_low_quality',
  'run_fname': 'dpo/dataset=AlpacLo/204234',
  'save_

In [5]:
# TODO add seed, train, other parts from config
def load_config(f):
    # print(f.parent.parent.stem)
    config = json.load((f.parent / 'config.json').open())
    # print(config)
    # log = (f.parent / 'log.txt').open().read()

    # dataframe of all non list/dict/tuple in config
    config_df = pd.DataFrame({k: v for k, v in config.items() if not isinstance(v, (list, dict, tuple))}, index=[0])
    # config_df['log'] = log
    config_df['file'] = f
    config_df['ts'] = f.parent.stem
    _, method, _ = f.parent.parent.stem.split('_', 2) # this is {model}_{method}_{dataset}
    config_df['method'] = method

    df = pd.read_parquet(f)
    df['seed'] = config['seed']
    df['train'] = config['dataset']
    df['base_model'] = config['base_model']
    df['ts'] = config_df['ts']
    for k in config:
        if isinstance(config[k], (list, tuple)):
            continue
        if isinstance(config[k], dict):
            for j in config[k]:
                if isinstance(j, (list, dict, tuple)):
                    continue
                # print(f'Adding {k}_{j} to df')
                df[f'{k}.{j}'] = config[k][j]
            continue
        df[k] = config[k]


    return df

df_res2 = pd.concat([load_config(f) for f in fs[1:]])
df_res2.columns

Index(['model', 'dataset', 'ds_i', 'correct', 'prob', 'score__sigmoid',
       'score__log_ratio', 'score__correct', 'score__chosen_log_score',
       'score__rejected_log_score', '_chosen_logps', '_rejected_logps',
       '_chosen_ppl', '_rejected_ppl', '_l_chosen', '_l_rejected', 'adapter',
       'seed', 'train', 'target', 'type', 'label', 'category', 'ds_name_nice',
       'base_model', 'ts', 'lr', 'weight_decay', 'gradient_clip_val',
       'ideal_batch_size', 'pl_precision', 'num_workers', 'verbose',
       'patience', 'dev', 'load_in_4bit', 'load_in_8bit', 'use_grad_paging',
       'n_samples', 'eval_samples', 'max_length', 'max_prompt_length',
       'batch_size', 'save', 'wandb', 'collection_layers', 'collect_input',
       'collect_hs', 'loss.trust_region', 'loss.α', 'loss.filter_sinks',
       'loss.eps', 'loss.β', 'loss.p', 'loss.use_policy_weights',
       'loss.inner_policy_weights', 'loss.align_method',
       'loss.norm_before_reduce', 'loss.dpo_agg_type', 'loss.clamp_b

In [6]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

In [7]:
from reprpo.data.util import sort_str, nice_ds_name

df_res2.columns

Index(['model', 'dataset', 'ds_i', 'correct', 'prob', 'score__sigmoid',
       'score__log_ratio', 'score__correct', 'score__chosen_log_score',
       'score__rejected_log_score', '_chosen_logps', '_rejected_logps',
       '_chosen_ppl', '_rejected_ppl', '_l_chosen', '_l_rejected', 'adapter',
       'seed', 'train', 'target', 'type', 'label', 'category', 'ds_name_nice',
       'base_model', 'ts', 'lr', 'weight_decay', 'gradient_clip_val',
       'ideal_batch_size', 'pl_precision', 'num_workers', 'verbose',
       'patience', 'dev', 'load_in_4bit', 'load_in_8bit', 'use_grad_paging',
       'n_samples', 'eval_samples', 'max_length', 'max_prompt_length',
       'batch_size', 'save', 'wandb', 'collection_layers', 'collect_input',
       'collect_hs', 'loss.trust_region', 'loss.α', 'loss.filter_sinks',
       'loss.eps', 'loss.β', 'loss.p', 'loss.use_policy_weights',
       'loss.inner_policy_weights', 'loss.align_method',
       'loss.norm_before_reduce', 'loss.dpo_agg_type', 'loss.clamp_b

In [8]:
from reprpo.data.util import df_sort_cols

In [9]:
for train_set, group in df_res2.groupby(['base_model', "train", 'n_samples']): # TODO also dev, n_samples
    print(train_set)
    # per-run means for each seed, dataset, adapter
    group['adapter2'] = group['adapter'] + '_' +  group['loss.align_method']
    run_stats = group.groupby(["seed", "ds_name_nice", "adapter2"])['correct'].mean().reset_index()
    stats = run_stats.groupby(["ds_name_nice", "adapter2"])['correct'].agg(['mean','std']).reset_index()
    mean_df = stats.pivot(index='adapter2', columns='ds_name_nice', values='mean')
    std_df = stats.pivot(index='adapter2', columns='ds_name_nice', values='std')
    # sort adapters by mean on first dataset column
    mean_df = mean_df.sort_values(by=mean_df.columns[0], axis=0)

    mean_df = df_sort_cols(mean_df, first=['base','none'])

    mean_df.index.name = 'adapter/ds'
    # combine mean and std into strings
    df_res_ds = mean_df.copy()
    for col in mean_df.columns:
        df_res_ds[col] = mean_df[col].combine(std_df[col], lambda m, s: f"{m:.3f}±{s:.3f}")
    display(df_res_ds)

    # per-run means for distribution shift types
    run_stats2 = group.groupby(["seed", "type", "adapter2"])['correct'].mean().reset_index()
    stats2 = run_stats2.groupby(["type", "adapter2"])['correct'].agg(['mean','std']).reset_index()


    mean2 = stats2.pivot(index='adapter2', columns='type', values='mean')
    std2 = stats2.pivot(index='adapter2', columns='type', values='std')

    mean2 = df_sort_cols(mean2, first=['in_domain'], last=['orthogonal'])
    mean2 = df_sort_cols(mean2.T, first=['base','none']).T

    mean2.index.name = 'adapter / distribution_shift'
    # combine mean and std into strings
    df_res_type = mean2.copy()
    for col in mean2.columns:
        df_res_type[col] = mean2[col].combine(std2[col], lambda m, s: f"{m:.3f}±{s:.3f}")
    display(df_res_type)

    # bar chart
    base_model = group['base_model'].iloc[0]
    if run_stats2.adapter2.nunique()>2:

        hue_order = sort_str(run_stats2.adapter2.tolist(), first=['none', 'dpo'], sort_middle=True, unique=True)
        order = sort_str(run_stats2.type.tolist(), first=['in_domain'], last=['orthogonal'], unique=True)

        sns.barplot(data=run_stats2, 
                    y='type', 
                    x='correct', hue='adapter2', 
                    errorbar='sd', orient='h', 
                    legend='full',
                    hue_order=hue_order,
                    order=order,
        )
        plt.title(f"{train_set}\nper-run means")
        plt.ylabel('distribution shift type')
        plt.xlabel('accuracy')
        plt.show() 
        # TODO maybe show deviation from base by plotting it underneatm and the top has alpha 0.5

('allenai/OLMo-2-0425-1B-SFT', 'math', np.int64(25000))


ds_name_nice,cross_domain (change_my_view_test ),cross_domain (math_fiction_test ),in_domain (math_test ),moral_transfer (ethics_justice_test ),moral_transfer (ethics_utilitarianism_test ),orthogonal (medical_dpo_v2_test_data )
adapter/ds,,,,,,
none_pars_rat,0.594±nan,0.812±nan,0.812±nan,0.438±nan,0.656±nan,0.375±nan
side-None-InnerDPO_pars_rat,0.594±nan,0.812±nan,0.812±nan,0.438±nan,0.656±nan,0.375±nan


type,in_domain,cross_domain,moral_transfer,orthogonal
adapter / distribution_shift,,,,
none_pars_rat,0.812±nan,0.703±nan,0.547±nan,0.375±nan
side-None-InnerDPO_pars_rat,0.812±nan,0.703±nan,0.547±nan,0.375±nan


('princeton-nlp/Llama-3-Base-8B-SFT', 'math', np.int64(25000))


ds_name_nice,cross_domain (change_my_view_test ),cross_domain (math_fiction_test ),in_domain (math_test ),moral_transfer (ethics_justice_test ),moral_transfer (ethics_utilitarianism_test ),orthogonal (medical_dpo_v2_test_data )
adapter/ds,,,,,,
none_pars_rat,0.531±nan,0.438±nan,0.812±nan,0.250±nan,0.375±nan,0.344±nan
side-None-InnerDPO_pars_rat,0.531±nan,0.438±nan,0.812±nan,0.250±nan,0.375±nan,0.344±nan


...

ds_name_nice,cross_domain (math_test ),cross_domain (raven_matrices_test ),in_domain (cooking_test ),moral_transfer (ethics_commonsense_test ),moral_transfer (ethics_justice_test ),orthogonal (medical_dpo_v2_test_data )
adapter/ds,,,,,,
hs-None-InnerDPO_pars_rat,0.845±nan,0.681±nan,0.935±nan,0.637±nan,0.456±nan,0.259±nan
none_pars_rat,0.923±nan,0.676±nan,0.903±nan,0.754±nan,0.553±nan,0.419±nan


type,in_domain,cross_domain,moral_transfer,orthogonal
adapter / distribution_shift,,,,
hs-None-InnerDPO_pars_rat,0.935±nan,0.763±nan,0.540±nan,0.259±nan
none_pars_rat,0.903±nan,0.799±nan,0.646±nan,0.419±nan


('wassname/Qwen3-0.6B-sft', 'math', np.int64(25000))


ds_name_nice,cross_domain (change_my_view_test ),cross_domain (math_fiction_test ),in_domain (math_test ),moral_transfer (ethics_justice_test ),moral_transfer (ethics_utilitarianism_test ),orthogonal (medical_dpo_v2_test_data )
adapter/ds,,,,,,
none_pars_rat,0.594±nan,0.844±nan,0.844±nan,0.469±nan,0.656±nan,0.406±nan
side-None-InnerDPO_pars_rat,0.594±nan,0.844±nan,0.844±nan,0.469±nan,0.656±nan,0.406±nan


type,in_domain,cross_domain,moral_transfer,orthogonal
adapter / distribution_shift,,,,
none_pars_rat,0.844±nan,0.719±nan,0.562±nan,0.406±nan
side-None-InnerDPO_pars_rat,0.844±nan,0.719±nan,0.562±nan,0.406±nan
